In [3]:
# portfolio_rebalance_animation.py

import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.optimize import minimize

# === Configuration ===
tickers = ['AAPL', 'WMT', 'TSLA', 'KO', 'BAC', 'T', 'META', 'NFLX', 'CRM']
start_date = '2020-01-01'
end_date = '2022-01-01'

# === Download Price Data ===
prices = yf.download(tickers, start=start_date, end=end_date)['Close'].dropna()

# === Strategy Functions ===
def equal_weight(returns, cov):
    return np.repeat(1 / returns.shape[1], returns.shape[1])

def inv_vol_weight(returns, cov):
    vol = returns.std()
    w = 1 / vol
    return (w / w.sum()).values

def gmv_weight(returns, cov):
    def objective(w): return w.T @ cov @ w
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, 1)] * len(cov)
    result = minimize(objective, np.ones(len(cov)) / len(cov), bounds=bounds, constraints=constraints)
    return result.x

def markowitz_weight(returns, cov):
    mean_ret = returns.mean() * 252
    def objective(w): return -((w @ mean_ret) / np.sqrt(w.T @ cov @ w))
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, 1)] * len(cov)
    result = minimize(objective, np.ones(len(cov)) / len(cov), bounds=bounds, constraints=constraints)
    return result.x

def risk_parity_weight(returns, cov):
    def risk_contrib(w):
        total = w @ cov @ w
        return w * (cov @ w) / total
    def objective(w):
        rc = risk_contrib(w)
        return np.sum((rc - rc.mean()) ** 2)
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, 1)] * len(cov)
    result = minimize(objective, np.ones(len(cov)) / len(cov), bounds=bounds, constraints=constraints)
    return result.x

# === Monthly Rebalancing Simulator ===
def rebalance_monthly(prices, strategy_func):
    monthly_returns = prices.pct_change().resample('ME').apply(lambda x: (1 + x).prod() - 1).dropna()
    dates = monthly_returns.index
    port_returns = []
    weights_over_time = []
    drawdowns = []
    risk_contribs = []
    cum_value = 1
    hwm = 1

    for i in range(len(dates) - 1):
        returns_window = prices[:dates[i]].pct_change().dropna()
        cov = returns_window.cov() * 252
        weights = strategy_func(returns_window, cov)
        weights_over_time.append((dates[i + 1], weights))

        month_ret = monthly_returns.iloc[i + 1]
        ret = weights @ month_ret.values
        cum_value *= (1 + ret)
        hwm = max(hwm, cum_value)
        drawdown = (hwm - cum_value) / hwm

        port_returns.append((dates[i + 1], cum_value))
        drawdowns.append((dates[i + 1], drawdown))

        rc = weights * (cov @ weights)
        rc /= rc.sum()
        risk_contribs.append((dates[i + 1], rc))

    return (
        pd.DataFrame(port_returns, columns=["Date", "Value"]).set_index("Date"),
        pd.DataFrame(weights_over_time, columns=["Date", "Weights"]).set_index("Date"),
        pd.DataFrame(drawdowns, columns=["Date", "Drawdown"]).set_index("Date"),
        pd.DataFrame(risk_contribs, columns=["Date", "RiskContrib"]).set_index("Date")
    )

# === Animate Portfolio Metrics Over Time ===
def animate_metric(all_data, value_col, title, y_label):
    df = pd.concat(all_data, names=["Strategy", "Date"]).reset_index()
    fig = px.line(df, x="Date", y=value_col, color="Strategy",
                  title=title, labels={value_col: y_label, "Date": "Date"})
    fig.show()

def animate_weights(all_weights_df, title="Portfolio Weights Over Time"):
    df = pd.concat(all_weights_df, names=["Strategy", "Date"])
    df = df.reset_index().explode("Weights")
    df["Ticker"] = tickers * (len(df) // len(tickers))
    df["Weight"] = df["Weights"]
    df.drop(columns="Weights", inplace=True)

    fig = px.bar(df, x="Ticker", y="Weight", animation_frame=df["Date"].astype(str),
                 title=title, range_y=[0, 1], color="Strategy", barmode="group")
    fig.update_layout(xaxis_tickangle=45)
    fig.show()

# === Run the Simulation ===
if __name__ == "__main__":
    strategies = {
        "Equal Weight": equal_weight,
        "Inverse Vol": inv_vol_weight,
        "GMV": gmv_weight,
        "Markowitz": markowitz_weight,
        "Risk Parity": risk_parity_weight
    }

    all_weights = {}
    all_returns = {}
    all_drawdowns = {}
    all_risk_contribs = {}

    for name, func in strategies.items():
        returns_df, weights_df, dd_df, rc_df = rebalance_monthly(prices, func)
        all_returns[name] = returns_df.rename(columns={"Value": "Cumulative Return"})
        all_weights[name] = weights_df
        all_drawdowns[name] = dd_df.rename(columns={"Drawdown": "Drawdown"})
        all_risk_contribs[name] = rc_df

    animate_weights(all_weights, title="Asset Weights Over Time (All Strategies)")
    # animate_metric(all_returns, value_col="Cumulative Return", title="Cumulative Returns Over Time", y_label="Growth of $1")
    # animate_metric(all_drawdowns, value_col="Drawdown", title="Drawdowns Over Time", y_label="Drawdown")


/var/folders/rf/w1c5pmc51gvg00xtlws6z9nh0000gn/T/ipykernel_24377/717541351.py:15: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  9 of 9 completed
